In [1]:
import numpy as np
from sympy import symbols, cos, sin, pi, simplify, sqrt, atan2, asin, acos
from sympy.matrices import Matrix

In [2]:
q1, q2, q3, q4, q5, q6, q7 = symbols('q1:8')
d1, d2, d3, d4, d5, d6, d7 = symbols('d1:8')
a0, a1, a2, a3, a4, a5, a6 = symbols('a0:7')
alpha0, alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = symbols('alpha0:7')

In [3]:
def rot_x(q):
    M1 = Matrix([[0],[0],[0]])
    M2 = Matrix([[0, 0, 0, 1]])
    Rx = Matrix([[  1,      0,       0],
                 [  0, cos(q), -sin(q)],
                 [  0, sin(q),  cos(q)]])
    rot_x = Rx.row_join(M1)
    rot_x = rot_x.col_join(M2)
    return rot_x
    
def rot_y(q):   
    M1 = Matrix([[0],[0],[0]])
    M2 = Matrix([[0, 0, 0, 1]])
    Ry = Matrix([[ cos(q),     0, sin(q)],
                 [      0,     1,      0],
                 [-sin(q),     0, cos(q)]])
    rot_y = Ry.row_join(M1)
    rot_y = rot_y.col_join(M2)
    return rot_y

def rot_z(q):   
    M1 = Matrix([[0],[0],[0]])
    M2 = Matrix([[0, 0, 0, 1]])
    Rz = Matrix([[ cos(q),-sin(q), 0],
                 [ sin(q), cos(q), 0],
                 [      0,      0, 1]])
    rot_z = Rz.row_join(M1)
    rot_z = rot_z.col_join(M2)
    return rot_z

def trans_x(d):
    M1 = Matrix(np.eye(3))
    M2 = Matrix([[d],[0],[0]])
    M3 = Matrix([[0, 0, 0, 1]])
    Trans_x = M1.row_join(M2)
    Trans_x = Trans_x.col_join(M3)
    return Trans_x

def trans_y(d):
    M1 = Matrix(np.eye(3))
    M2 = Matrix([[0],[d],[0]])
    M3 = Matrix([[0, 0, 0, 1]])
    Trans_y = M1.row_join(M2)
    Trans_y = Trans_y.col_join(M3)
    return Trans_y

def trans_z(d):
    M1 = Matrix(np.eye(3))
    M2 = Matrix([[0],[0],[d]])
    M3 = Matrix([[0, 0, 0, 1]])
    Trans_z = M1.row_join(M2)
    Trans_z = Trans_z.col_join(M3)
    return Trans_z

def DH_transform(q1, d1, alpha0, a0):
    '''The modified DH convention transform matrix
    alpha0: twist angle, a0: link length, 
    d1: link offset, q1: joint angle'''
    T0_1 = Matrix([[cos(q1), -sin(q1),      0,    a0],
                   [sin(q1) * cos(alpha0), cos(q1) * cos(alpha0), -sin(alpha0), -sin(alpha0) * d1],
                   [sin(q1) * sin(alpha0), cos(q1) * sin(alpha0),  cos(alpha0),  cos(alpha0) * d1],
                   [0,              0,      0,     1]])
    return T0_1
    

In [4]:
# The Modified DH params
s = {alpha0:       0, a0:      0, d1:  0.75,
     alpha1: -pi / 2, a1:   0.35, d2:     0, q2: q2 - pi / 2,
     alpha2:       0, a2:   1.25, d3:     0,
     alpha3: -pi / 2, a3: -0.054, d4:   1.5,
     alpha4:  pi / 2, a4:      0, d5:     0,
     alpha5: -pi / 2, a5:      0, d6:     0,
     alpha6:       0, a6:      0, d7: 0.303, q7: 0}

In [5]:
# Define Modified DH Transformation matrix
# base_link to link1
T0_1 = DH_transform(q1, d1, alpha0, a0)
T0_1 = T0_1.subs(s)
# linke1 to link 2
T1_2 = DH_transform(q2, d2, alpha1, a1)
T1_2 = T1_2.subs(s)
# link2 to link3
T2_3 = DH_transform(q3, d3, alpha2, a2)
T2_3 = T2_3.subs(s)
# link3 to link4
T3_4 = DH_transform(q4, d4, alpha3, a3)
T3_4 = T3_4.subs(s)
# link4 to link5
T4_5 = DH_transform(q5, d5, alpha4, a4)
T4_5 = T4_5.subs(s)
# link5 to link6
T5_6 = DH_transform(q6, d6, alpha5, a5)
T5_6 = T5_6.subs(s)
# link6 to end-effector
T6_7 = DH_transform(q7, d7, alpha6, a6)
T6_7 = T6_7.subs(s)

In [6]:
# Create individual transformation matrices
T0_2 = simplify(T0_1 * T1_2)
T0_3 = simplify(T0_2 * T2_3)
T0_4 = simplify(T0_3 * T3_4)
T0_5 = simplify(T0_4 * T4_5)
T0_6 = simplify(T0_5 * T5_6)
T0_G = simplify(T0_6 * T6_7)

R_corr = rot_z(pi) * rot_y(-pi/2)
T_total = simplify(T0_G * R_corr)

In [7]:
import pickle
import os
if not os.path.exists("T_final.p"):
    pickle.dump(T0_1, open("T0_1.p", "wb"))
    pickle.dump(T0_2, open("T0_2.p", "wb"))
    pickle.dump(T0_3, open("T0_3.p", "wb"))
    pickle.dump(T0_4, open("T0_4.p", "wb"))
    pickle.dump(T0_5, open("T0_5.p", "wb"))
    pickle.dump(T0_6, open("T0_6.p", "wb"))
    pickle.dump(T0_G, open("T0_G.p", "wb"))
    pickle.dump(T_total, open("T_final.p", "wb"))
else:
    T0_1 = pickle.load(open( "T0_1.p", "rb" ) )
    T0_2 = pickle.load(open( "T0_2.p", "rb" ) )
    T0_3 = pickle.load(open( "T0_3.p", "rb" ) )
    T0_4 = pickle.load(open( "T0_4.p", "rb" ) )
    T0_5 = pickle.load(open( "T0_5.p", "rb" ) )
    T0_6 = pickle.load(open( "T0_6.p", "rb" ) )
    T0_G = pickle.load(open( "T0_G.p", "rb" ) )
    T_total = pickle.load(open( "T_total.p", "rb" ) )

In [8]:
# # Numerically evaluate transforms
state = {q1:	-1.06, q2:	-0.54, q3: 0.94, 
         q4: 	-2.97, q5:	0.05, q6:	4.16}
# # print("T0_1 = ", T0_1.evalf(subs=state))
# # print("T0_2 = ", T0_2.evalf(subs=state))
# # print("T0_3 = ", T0_3.evalf(subs=state))
# # print("T0_4 = ", T0_4.evalf(subs=state))
# print("T0_5 = ", T0_5.evalf(subs=state))
# print("T0_6 = ", T0_6.evalf(subs=state))
# print("T0_G = ", T0_G.evalf(subs=state))
print("T_total = ", T_total.evalf(subs=state))

T_total =  Matrix([
[ 0.451648647906469,   0.481759752870882, -0.750946761999841, 0.658912570897484],
[ -0.82338964238947, -0.0990529698130237, -0.558764714327024, -1.18106929707144],
[-0.343573857634714,   0.870687113540224,  0.351938850746823,  1.08416816570233],
[                 0,                   0,                  0,               1.0]])


In [9]:
# gripper position from FK calculation
position_fk = T_total.evalf(subs=state)[0:3,3]
# gripper position from rviz simulator
position_g = Matrix([[0.65841], [-1.1795], [1.0837]])

In [10]:
position_error = position_fk - position_g
print("Calculation error: ", position_error)

Calculation error:  Matrix([
[0.000502570897483623],
[-0.00156929707144005],
[0.000468165702330214]])


In [11]:
# Position error
position_error.norm()

0.00171302362221487

### Inverse Kinematics

In [12]:
def rot_roll(q):
    R_x = Matrix([[      1,      0,      0],
                  [      0, cos(q), -sin(q)],
                  [      0, sin(q),  cos(q)]])
    return R_x
    
def rot_pitch(q):              
    R_y = Matrix([[ cos(q),     0, sin(q)],
                  [      0,     1,      0],
                  [-sin(q),     0, cos(q)]]) 
    return R_y

def rot_yaw(q):    
    R_z = Matrix([[ cos(q),-sin(q), 0],
                  [ sin(q), cos(q), 0],
                  [      0,      0, 1]])  
    return R_z

In [13]:
r, p, y = symbols("r p y") # end-effector orientation
R0_g_symp = simplify(rot_yaw(y)* rot_pitch(p) * rot_roll(r))
R_corr = rot_yaw(pi) * rot_pitch(-pi/2)

In [14]:
# px, py, pz = 2.1529, 0, 1.9465 # initial position
px, py, pz = 0.49792, 1.3673, 2.4988
roll, pitch, yaw = 0.366, -0.078, 2.561
# roll, pitch, yaw =0,0,0
pg_0 = Matrix([[px],[py],[pz]])
R0_g = R0_g_symp.evalf(subs={r: roll, p: pitch, y: yaw})

### Wrist center and theta1

In [15]:
pwc_0 = simplify(pg_0 - (0.303) * R0_g[0:3, 0:3]  * R_corr * Matrix([[0],[0],[1]]))
pwc_0

Matrix([
[0.750499428337951],
[ 1.20160389781975],
[ 2.47518995758694]])

In [20]:
p_link5 = Matrix([[0.75044],[1.2049],[2.4576]])

In [21]:
p5_error = pwc_0 - p_link5
p5_error.norm()

0.0178962127060553

In [22]:
theta1 = atan2(pwc_0[1], pwc_0[0])
np.float64(theta1)

1.0124980936377144

### theta3 and theta2

In [23]:
p2_sym = T0_2 * Matrix([0,0,0,1])
p2_0 = p2_sym.evalf(subs={q1: theta1})
pwc_2 = pwc_0 - p2_0[0:3,:]

In [24]:
l23 = a2
l35 = sqrt(a3**2 + d4**2)
p25 = sqrt(np.sum(np.square(pwc_2)))

In [25]:
theta3_1 = atan2(a3,d4)
c235 = (np.sum(np.square(pwc_2)) - l23**2 - l35**2) / (2*l23*l35)
theta3_2 = atan2(sqrt(1-c235**2), c235)
# theta3_phi = atan2(-sqrt(1-c235**2), c235)
theta3 = (theta3_2 + theta3_1 - pi/2).subs(s)
np.float64(theta3)

-0.11568665105374737

In [26]:
theta2_out = atan2(pwc_2[2], sqrt(pwc_2[0]**2 + pwc_2[1]**2))
c523 = (-l35**2 + l23**2 + p25**2) / (2*l23 * p25)
theta2_phi = atan2(sqrt(1 - c523**2), c523)
theta2 = (pi/2 - (theta2_phi + theta2_out)).subs(s)
np.float64(theta2)

-0.2758003637377226

### theta5, 4, 6

In [27]:
def Euler_angles_from_matrix_URDF(R):
            ''' Calculate q4-6 from R3_6 rotation matrix
            Input R is 3x3 rotation matrix, output are Euler angles :q4, q5, q6'''
            r12, r13 = R[0,1], R[0,2]
            r21, r22, r23 = R[1,0], R[1,1], R[1,2] 
            r32, r33 = R[2,1], R[2,2]
            # # Euler angles from rotation matrix
            # q5 = atan2(sqrt(r13**2 + r33**2), r23)
            # q4 = atan2(r33, -r13)
            # q6 = atan2(-r22, r21)
            if np.abs(r23) is not 1:
                q5 = atan2(sqrt(r13**2 + r33**2), r23)
                q4 = atan2(r33, -r13)
                q6 = atan2(-r22, r21)
            else:
                q6 = 0
                if r23 == -1:
                    q5 = 0
                    q4 = q6 + atan2(-r12, -r32)
                else:
                    q5 = 0
                    q4 = -q6 + atan2(r12, r32)

            return np.float64(q4), np.float64(q5), np.float64(q6)

In [28]:
R0_3 = T0_3[0:3, 0:3]
R0_3_inv = simplify(R0_3 ** -1)

In [29]:
R0_3_inv_value = R0_3_inv.evalf(subs={q1:theta1, q2:theta2, q3:theta3})
R3_6 =  R0_3_inv_value * R0_g * R_corr
R3_6

Matrix([
[ 0.724593522747684,  -0.686240627701868, 0.0635454772855438],
[ 0.684432983034957,   0.727340296077276, 0.0502750975741564],
[-0.080720000780301, 0.00706361051918319,  0.996711787268748]])

In [30]:
theta4, theta5, theta6 = Euler_angles_from_matrix_URDF(R3_6)

In [31]:
theta4, theta5, theta6

(1.6344652724032285, 1.5205000259943031, -0.81578130619968381)

In [32]:
# # Numerically evaluate transforms
state = {q1:	theta1, q2:	theta2, q3: theta3, 
         q4: 	theta4, q5:	theta5, q6:	theta6}
print("T_total = ", T_total.evalf(subs=state))

T_total =  Matrix([
[-0.833595473062543,  -0.48887208255965, 0.257143295038827, 0.49792],
[  0.54685182237706, -0.796053601157403, 0.259329420712765,  1.3673],
[0.0779209320563016,  0.356795110642117,  0.93092726749696,  2.4988],
[                 0,                  0,                 0,     1.0]])
